In [57]:
import re
from datetime import datetime

# регулярное выражение для парсинга лог строки
log_pattern = r'(\S+) (\d+) (\S+) (\S+) spcd: \[(\S+)\] \[(\S+) (\S+)\] INFO: (.*)'

# регулярные выражения для парсинга INFO части
info_patterns = {
    'Incoming Connection' : r'Incoming Conn\{(\S+)\} on (\S+) (\S+), (\d+) of (\d+)',
    'On Connection'       : r'On Conn\{(\S+)\} new Query\{(\S+)\} \[(\S+)\]: (.*)',
    'End Query'           : r'End Query\{(\S+)\} \[(\S+)\], (\S+), (\S+) \{ (\S+) : (\S+) (\S+), (\S+) (\S+) \} (\S+), (\d+) (\S+)',
    'Query'               : r'Query\{(\S+)\} \[(\S+)\] full text: (\S+)',
    'Sending Query'       : r'Sending Query\{(\S+)\} \[(\S+)\] to (\S+)\{(\S+)\}'
}


def InfoParsing(inp: str) -> None|list:
    '''
    автоматически парсит cтроку с часть после "INFO:" с нужным регулярным выражением и возвращает тип запроса и данные запроса, d byjv ckexftr ybxtuj yt djpdhfoftn
    '''
    for data_type, pattern in info_patterns.items():
        match_info = re.match(pattern, inp)
        if match_info:
            return [data_type, match_info.groups()]

In [ ]:
ips = set() # мнодество ip адресов (множество так как проверка наличия и добавления адреса происходит за O(1)б ну и соответвенно нет повторов)
query =  dict() # счетчик количества для каждого запроса

cnt_query = 0 # счетчик полного числа запросов
total_words = 0 # полная сумма слов запросов для для нахождления среднего количества слов

max_response_time = 0 # максимальное время выпполнения запроса: ожидание в очереди + исполнение
total_response_time = 0 # полное время выпполнения запроса для нахождения среднего времени
max_n_query = 0 # счетчик максималного числа запроса для нахождения самого популярного запроса

log = open( "spcd.log.11", "r" , encoding="utf8")
for k, log_entry in enumerate(log):
    match = re.match(log_pattern, log_entry)
    url = match.group(4)
    date_time = datetime.strptime(match.group(6)[:-2] + '20'+ match.group(6)[-2:] + ' ' + match.group(7), '%d.%m.%Y %H:%M:%S')
    info  = match.group(8)
    if k == 0:
        date_time_0 = date_time
    info_data = InfoParsing(info)
    
    if not info_data:
        continue 
    if info_data[0]=='Incoming Connection':
        con_id = info_data[1][0]
        ip = info_data[1][1].split(':')[0]
        ips.add(ip)

    elif info_data[0]=='On Connection':
        cnt_query += 1
        con_id = info_data[1][0]
        query_id = info_data[1][1]
        person_id = info_data[1][2]
        search_phrase_words = info_data[1][3]

        if search_phrase_words in query.keys():
            query[search_phrase_words]+=1
        else:
            query.update({search_phrase_words:1})

        if query[search_phrase_words] > max_n_query:
            most_popular_query = search_phrase_words
            max_n_query = query[search_phrase_words]

        cnt_words = len(search_phrase_words.split())
        total_words += cnt_words
        

    elif info_data[0]=='End Query':
        query_id = info_data[1][0]
        person_id = info_data[1][1]
        response_time = float(info_data[1][4])
        total_response_time += response_time
        if response_time > max_response_time:
            max_response_time = response_time

log.close()

average_reponse_time = total_response_time/cnt_query
average_num_words = total_words/cnt_query
average_num_queres_per_sec = cnt_query / (date_time-date_time_0).total_seconds()

In [58]:
print(f"Самый популярный запрос: \t\t{most_popular_query}")
print(f"Среднее количество слов в запросах: \t{average_num_words:.2f}")
print(f"Среднее время обработки запроса: \t{average_reponse_time:.2f}")
print(f"Максимальное время обработки запроса: \t{max_response_time:.2f}")
print(f"Среднее количесво запросов в секунду: \t{average_num_queres_per_sec:.2f}")

print("\nСписок всех клиентов сервиса (ip-адреса):")
print(*ips, sep='\n')

Самый популярный запрос: 		how to make a sandwich
Среднее количество слов в запросах: 	2.65
Среднее время обработки запроса: 	1.07
Максимальное время обработки запроса: 	45.79
Среднее количесво запросов в секунду: 	1780.17

Список всех клиентов сервиса (ip-адреса):
134.0.0.1
10.205.6.47434
10.205.6.47310
10.205.6.47272
10.205.6.47496
10.200.243.75
10.205.6.47300
10.205.6.47494
10.205.6.47764
10.205.6.47560
10.205.6.47102
10.205.6.47820
10.205.6.47128
10.205.6.47398
10.205.6.47134
10.205.6.47592
10.205.6.47718
10.205.6.47230
10.205.6.47576
10.205.6.47646
10.205.6.47390
10.205.6.47520
10.205.6.47660
10.205.6.47306
10.205.6.47698
10.205.6.47558
10.205.6.47736
10.205.6.130
10.205.6.47330
10.205.6.47750
10.205.6.47320
10.205.6.47440
133.0.0.1
10.205.6.134
10.205.6.47212
10.205.6.47720
10.205.6.47804
10.205.6.47446
10.205.6.132
10.205.6.47040
10.205.6.47166
10.205.6.47070
10.205.6.47632
10.205.6.47092
10.205.6.47784
10.205.6.47014
10.205.6.47256
10.205.6.47186
10.205.6.47824
10.205.6.47366
1